In [2]:
import numpy as np
import pandas as pd
import numpy.random as rd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import model_selection

In [71]:
from pycorenlp import StanfordCoreNLP

In [14]:
df = pd.read_csv(r"C:\Users\tilwa\majorproject\data\sentimental_analysis\teslastocktweets.csv")

In [15]:
df.head()

,id,created_at,date,time,user_id,tweet,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116
0,1.382578e+18,2021-04-15 11:42:20 India Standard Time,2021-04-15,11:42:20,4.235102e+09,@tokenmetricsinc Tokenised stocks like the Tes...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.382573e+18,2021-04-15 11:23:23 India Standard Time,2021-04-15,11:23:23,9.376913e+17,$BTC hundreds of funds have invested in publi...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.382573e+18,2021-04-15 11:22:50 India Standard Time,2021-04-15,11:22:50,3.736926e+09,nkakatuwa. nagtry ako bumili ng stocks ng tesl...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.382563e+18,2021-04-15 10:44:08 India Standard Time,2021-04-15,10:44:08,3.113506e+09,@MissesJackson @GadzooksB @AP @elonmusk @jack ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.382561e+18,2021-04-15 10:37:33 India Standard Time,2021-04-15,10:37:33,9.159254e+07,@austinmcraig @eugenelee3 @nntaleb If you boug...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5497 entries, 0 to 5496
Columns: 117 entries, id to Unnamed: 116
dtypes: float64(2), object(115)
memory usage: 4.9+ MB


In [29]:
df1 = df[["date", "tweet"]]
df1

,date,tweet
0,2021-04-15,@tokenmetricsinc Tokenised stocks like the Tes...
1,2021-04-15,$BTC hundreds of funds have invested in publi...
2,2021-04-15,nkakatuwa. nagtry ako bumili ng stocks ng tesl...
3,2021-04-15,@MissesJackson @GadzooksB @AP @elonmusk @jack ...
4,2021-04-15,@austinmcraig @eugenelee3 @nntaleb If you boug...
...,...,...
5492,2021-03-20,#doge #dogecoin $doge #dogeto10cents #dogetoth...
5493,2021-03-20,Perfect response to those criticizing @elonmus...
5494,2021-03-20,$TSLA closes extended hours -0.40%
5495,2021-03-20,Examples of growth stocks: ⁃Tesla ⁃Amazon ⁃...


In [37]:
df2 = df1.loc[(df1["date"] > '2021-03-20') & (df1["date"] < '2021-03-31')]

In [67]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2039 entries, 3383 to 5421
Data columns (total 2 columns):
date     2039 non-null datetime64[ns]
tweet    2014 non-null object
dtypes: datetime64[ns](1), object(1)
memory usage: 47.8+ KB


In [73]:
Row_list =[] 
#Row_list1 = []

# Iterate over each row 
for index, rows in df2.iterrows(): 
    # Create list for the current row 
    my_list =[rows['date'], rows['tweet']] 
    
    # append the list to the final list 
    Row_list.append(my_list)
    
print(Row_list[:10])
print(len(Row_list))

[[Timestamp('2021-03-30 00:00:00'), '✨ Participate in the April #Webull wheel event to win awesome prizes! 😍 4000 shares of TSLA stocks are waiting to be claimed. 💙  https://t.co/rzd3Mk1zpq  Just saying'], [Timestamp('2021-03-30 00:00:00'), '$tsla 619$ need clear break 633/639*** #trading #stocks #Tesla  https://t.co/FBtzkJLs1B'], [Timestamp('2021-03-30 00:00:00'), '✨ Participate in the April #Webull wheel event to win awesome prizes! 😍 4000 shares of TSLA stocks are waiting to be claimed. 💙  https://t.co/g79sQHTTG8 #tesla #Stocks  https://t.co/PaBRdPVMYc'], [Timestamp('2021-03-30 00:00:00'), '4k Tesla!? ✨ Participate in the April #Webull wheel event to win awesome prizes! 😍 4000 shares of TSLA stocks are waiting to be claimed. 💙  https://t.co/dGzNe9i8mF'], [Timestamp('2021-03-30 00:00:00'), 'Dow'], [Timestamp('2021-03-30 00:00:00'), 'Show some proof otherwise. How much in fees has Tesla paid to Wall St?  How many times have their year out earnings estimates not been too high?  What do

In [93]:
ts = pd.Timestamp(year = 2021,  month = 3, day = 21)
tweet_per_day = []

for i in Row_list:
    if(i[0] == ts):
        tweet_per_day.append(i[1])
        
print(tweet_per_day)       
nlp = StanfordCoreNLP('http://localhost:9000')
results = nlp.annotate(tweet_per_day,properties={
        'annotators':'sentiment, ner, pos',
        'outputFormat': 'json',
        'timeout': 50000,
        })

sentimentList = []

for s in results["sentences"]:
    sentimentList.append(int(s["sentimentValue"]))
    #print("%s" % (s["sentimentValue"]))
    
print(sentimentList)



['@Jevaughn_Brown That’s true. Few people believed Tesla would make it to their current valuation. All the shorts got slaughtered lmao  But it definitely seems like most stock valuations are completely divorced from earnings. 10+ years of future growth priced into all these tech stocks. Crazy', '$TSLA to $3k? 😍❤️📈 #ARKinvest   #StockMarket #Tesla #stocks #cathiewood  https://t.co/0QPNSFssAw', '@YangGangWA @BernieSanders I don’t understand why you hate Elon so much', '@blockjayne1 @EndgameXrp The people that holds Stocks thinks the same. Why people didn’t buy Apple', 'Zero-Emissions Vehicles Are Bringing Disruption to the $1.5 Trillion Trucking Market. Watch These Stocks', '@ttmygh Their new targets scream obvious ponzi / securities fraud / manipulation &amp', 'Where do u think is $tsla going this week?  #TSLA #trade #stocks #StockMarket #OptionsTrading #ElonMusk #cryptocurrencies #CryptoNews #BTC #doge #NASDAQ #TRADINGTIPS #TradingView #DayTrading #Daytrader #Stock #StocksToBuy #SpaceX

AssertionError: 